<a href="https://colab.research.google.com/github/kinan-02/Fintech/blob/main/LSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from tqdm import tqdm
import random
# Load the Parquet file

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)
    torch.cuda.manual_seed_all(0)


torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

df = pd.read_parquet('final_data.parquet')
df_test = pd.read_parquet('test_data.parquet')


features = df[['open_btc', 'high_btc', 'low_btc', 'trade_count_btc',
               'open_eth', 'high_eth', 'low_eth', 'close_eth', 'trade_count_eth',
               'open_cl', 'close_cl', 'high_cl', 'low_cl', 'trade_count_cl',
               'open_gold', 'close_gold', 'high_gold', 'low_gold', 'trade_count_gold',
               'open_uup', 'close_uup', 'high_uup', 'low_uup', 'trade_count_uup',
               'open_mstr', 'close_mstr', 'high_mstr', 'low_mstr', 'trade_count_mstr',
               'open_spy', 'close_spy', 'high_spy', 'low_spy', 'trade_count_spy',
               'open_ndaq', 'close_ndaq', 'high_ndaq', 'low_ndaq', 'trade_count_ndaq']]

test_features = df_test[['open_btc', 'high_btc', 'low_btc', 'trade_count_btc',
               'open_eth', 'high_eth', 'low_eth', 'close_eth', 'trade_count_eth',
               'open_cl', 'close_cl', 'high_cl', 'low_cl', 'trade_count_cl',
               'open_gold', 'close_gold', 'high_gold', 'low_gold', 'trade_count_gold',
               'open_uup', 'close_uup', 'high_uup', 'low_uup', 'trade_count_uup',
               'open_mstr', 'close_mstr', 'high_mstr', 'low_mstr', 'trade_count_mstr',
               'open_spy', 'close_spy', 'high_spy', 'low_spy', 'trade_count_spy',
               'open_ndaq', 'close_ndaq', 'high_ndaq', 'low_ndaq', 'trade_count_ndaq']]


target = df['close_btc'].shift(-1)
target_test = df_test['close_btc']

features = features[:-1]
target = target[:-1]
test_features = test_features[:-1]
target_test = target_test[:-1]


scaler = MinMaxScaler()
features = scaler.fit_transform(features)
test_features = scaler.transform(test_features)


X_train, X_test = features, test_features
y_train, y_test = target, target_test


X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)


train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


class LSTM_Transformer(nn.Module):
    def _init_(self, n_layers, tr_layers, n_heads_first, n_heads_second, input_dim, hidden_dim, output_dim, dropout):
        super()._init_()
        self.n_layers = n_layers
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.tr_layers = tr_layers
        self.n_heads_first = n_heads_first
        self.n_heads_second = n_heads_second
        self.input_fc = nn.Sequential(
            nn.Linear(input_dim, input_dim * 2),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(input_dim * 2, hidden_dim),
            nn.Dropout(dropout),
            nn.ReLU()
        )


        self.main_task = nn.LSTM(input_size=hidden_dim,
                                 hidden_size=hidden_dim,
                                 batch_first=True,
                                 num_layers=n_layers,
                                 dropout=dropout)

        self.output_fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, output_dim)
        )


    def generate_mask(self, size):
        """
        Generate a causal mask for the Transformer, preventing future positions from attending to earlier positions.
        """
        mask = torch.triu(torch.ones(size, size) * float('-inf'), diagonal=1)
        return mask

    def forward(self, input_vec):

        lstm_input = self.input_fc(input_vec)

        final_output, _ = self.main_task(lstm_input)
        output = self.output_fc(final_output)

        return output.squeeze()

def evaluate_model(model):
    model.eval()
    predictions = []
    actuals = []
    total_loss = 0
    total_percentage_loss = 0
    total_mape = 0
    loss_fn = torch.nn.MSELoss()

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            output = model(X_batch)
            loss = loss_fn(output, y_batch)
            total_loss += loss.item()
            percentage_loss = torch.sqrt(loss) / (torch.mean(y_batch) + 1e-10) * 100
            total_percentage_loss += percentage_loss.item()
            mape = torch.mean(torch.abs((y_batch - output) / (y_batch + 1e-10))) * 100
            total_mape += mape.item()
            predictions.append(output.numpy())
            actuals.append(y_batch.numpy())
    predictions[-1] = np.repeat(predictions[-1], 32)
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    predicted_returns = (predictions[1:] - predictions[:-1]) / predictions[:-1]
    real_returns = (actuals[1:] - actuals[:-1]) / actuals[:-1]
    predicted_var = calculate_var(predicted_returns, confidence_level=90)
    real_var = calculate_var(real_returns, confidence_level=90)

    print(f"Test Loss: {total_loss / len(test_loader):.6f}")
    print(f"Test Loss as Percentage: {total_percentage_loss / len(test_loader):.2f}%")
    print(f"Predicted VaR (95% confidence): {predicted_var}")
    print(f"Real VaR (95% confidence): {real_var}")
    print(f"Mean Absolute Percentage Error (MAPE): {total_mape / len(test_loader):.2f}%")
    return predictions, actuals

def train_model():

    model = LSTM_Transformer(
        n_layers=1, tr_layers=1, n_heads_first=4, n_heads_second=1,
        input_dim=X_train.shape[1], hidden_dim=32, output_dim=1, dropout=0
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0008)
    loss_fn = torch.nn.MSELoss()

    for epoch in range(15):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{30}', leave=False)
        for X_batch, y_batch in progress_bar:
            optimizer.zero_grad()

            output = model(X_batch)
            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix({'loss': total_loss / len(train_loader)})
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")

    return model


def calculate_var(returns, confidence_level=90):
    """
    Calculate Value at Risk (VaR) for a given set of returns and confidence level.
    """
    return np.percentile(returns, 100 - confidence_level)

trained_model = train_model()

pred, actuals = evaluate_model(trained_model)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1, Loss: 512258696.3510183


Epoch 2, Loss: 62788.518012466695


Epoch 3, Loss: 11108.828988024105


Epoch 4, Loss: 9724.647349790881


...

Epoch 11, Loss: 6409.467191107721


Epoch 12, Loss: 6318.096300562213


Epoch 13, Loss: 5784.610060838906


Epoch 14, Loss: 5858.068744766244


Epoch 15, Loss: 5644.338383316089
Test Loss: 4162.417426
Test Loss as Percentage: 0.09%
Predicted VaR (95% confidence): -0.0008045249851420522
Real VaR (95% confidence): -0.0006554247112944722
Mean Absolute Percentage Error (MAPE): 0.06%
